In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns",10) #設定pandas最多顯示出50個欄位資訊
pd.set_option('display.max_rows', None)

df =  pd.read_table("testing_label.txt")
df.rename(columns={"0#####don ' t you hate it when you hang on to a seemingly interesting movie to see the ending only to find out that the ending sucks ?":'文字'}, inplace = True)
new=pd.DataFrame({'文字':"0#####don ' t you hate it when you hang on to a seemingly interesting movie to see the ending only to find out that the ending sucks ?",
                  },
                 index=[1]) 
df=df.append(new,ignore_index=True)   # ignore_index=True,表示不按原来的索引，从0开始自动递增
df['變數']=df['文字'].str.split("#####").str.get(0).str.title()
df['字串']=df['文字'].str.split("#####").str.get(1).str.title()
df.head() #test

,文字,變數,字串
0,0#####ok so never went to the movies because f...,0,Ok So Never Went To The Movies Because Friend ...
1,1#####can ' t wait to see diversity ' s perfor...,1,Can ' T Wait To See Diversity ' S Performance !
2,1#####i love britney spears haha joey this is ...,1,I Love Britney Spears Haha Joey This Is What U...
3,1#####1 more day !,1,1 More Day !
4,0#####hates being this burnt !! ouch,0,Hates Being This Burnt !! Ouch


In [2]:
Y_test = df['變數']
Y_test=Y_test.astype(np.float64)

In [3]:
df1 =  pd.read_table("training_label.txt")

df1.rename(columns={"1 +++$+++ are wtf ... awww thanks !":'文字'}, inplace = True)
new=pd.DataFrame({'文字':"1 +++$+++ are wtf ... awww thanks !",
                  },
                 index=[1]) 
df1=df1.append(new,ignore_index=True)
train =df1.sample(10000)
training=train.reset_index(drop=True)
training['變數']=training['文字'].str.split('+').str.get(0).str.title()
training['字串']=training['文字'].str.split('+').str.get(6).str.title()
y=training['變數']
y=y.astype(np.float64)
training.head()


,文字,變數,字串
0,1 +++$+++ hands up who thinks axl rose is gay ...,1,Hands Up Who Thinks Axl Rose Is Gay Not That ...
1,0 +++$+++ you have the flu ? is just getting o...,0,You Have The Flu ? Is Just Getting Over It He...
2,0 +++$+++ can ' t sleep cause i feel like i ' ...,0,Can ' T Sleep Cause I Feel Like I ' M Going T...
3,0 +++$+++ i just bought a copy of the sun . i ...,0,I Just Bought A Copy Of The Sun . I Feel Dirty .
4,0 +++$+++ i think i am just destined to wear h...,0,I Think I Am Just Destined To Wear High Water...


In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
token = Tokenizer(num_words=3800) 
token.fit_on_texts(training['字串'])  
token.word_index
x_train_seq = token.texts_to_sequences(training['字串'])
x_test_seq = token.texts_to_sequences(df['字串'])
x_train = sequence.pad_sequences(x_train_seq, maxlen=380)
x_test = sequence.pad_sequences(x_test_seq, maxlen=380)


Using TensorFlow backend.


In [5]:
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN

In [25]:
modelRNN = Sequential()  
modelRNN.add(Embedding(output_dim=32,   #輸出的維度是32，希望將數字list轉換為32維度的向量
     input_dim=3800,  #輸入的維度是3800，也就是我們之前建立的字典是3800字
     input_length=380)) #數字list截長補短後都是380個數字


In [26]:
modelRNN.add(SimpleRNN(units=16))
modelRNN.add(Dense(units=256,activation='relu')) 
#modelRNN.add(Dropout(0.7))
modelRNN.add(Dense(units=1,activation='sigmoid'))
modelRNN.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 380, 32)           115200    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 16)                784       
_________________________________________________________________
dense_4 (Dense)              (None, 256)               4352      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 120,593
Trainable params: 120,593
Non-trainable params: 0
_________________________________________________________________


In [27]:
modelRNN.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy']) 

In [28]:
train_history = modelRNN.fit(x_train,y,epochs=10,batch_size=100,verbose=2,validation_split=0.2)#

C:\Users\HuanJian\Anaconda3\envs\tensorflow_gpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8000 samples, validate on 2000 samples
Epoch 1/10
 - 25s - loss: 0.6775 - accuracy: 0.5619 - val_loss: 0.6219 - val_accuracy: 0.6475
Epoch 2/10
 - 25s - loss: 0.5079 - accuracy: 0.7531 - val_loss: 0.5454 - val_accuracy: 0.7235
Epoch 3/10
 - 25s - loss: 0.3503 - accuracy: 0.8472 - val_loss: 0.5977 - val_accuracy: 0.7145
Epoch 4/10
 - 26s - loss: 0.2105 - accuracy: 0.9233 - val_loss: 0.7799 - val_accuracy: 0.7000
Epoch 5/10
 - 26s - loss: 0.1101 - accuracy: 0.9617 - val_loss: 1.0923 - val_accuracy: 0.6630
Epoch 6/10
 - 27s - loss: 0.0649 - accuracy: 0.9783 - val_loss: 1.2329 - val_accuracy: 0.6690
Epoch 7/10
 - 26s - loss: 0.0340 - accuracy: 0.9895 - val_loss: 1.4430 - val_accuracy: 0.6650
Epoch 8/10
 - 27s - loss: 0.0199 - accuracy: 0.9942 - val_loss: 1.6212 - val_accuracy: 0.6695
Epoch 9/10
 - 28s - loss: 0.0133 - accuracy: 0.9961 - val_loss: 1.7481 - val_accuracy: 0.6660
Epoch 10/10
 - 28s - loss: 0.0109 - accuracy: 0.9964 - val_loss: 1.8302 - val_accuracy: 0.6715


In [29]:
scores = modelRNN.evaluate(x_test, Y_test,verbose=1)
scores[1]

90/90 [==============================] - 0s 5ms/step


0.7777777910232544

In [33]:
token = Tokenizer(num_words=3800) 
token.fit_on_texts(training['字串'])  
token.word_index
x_train_seq = token.texts_to_sequences(training['字串'])
x_test_seq = token.texts_to_sequences(df['字串'])
x_train = sequence.pad_sequences(x_train_seq, maxlen=380)
x_test = sequence.pad_sequences(x_test_seq, maxlen=380)


In [6]:
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM

modelLSTM = Sequential() #建立模型
modelLSTM.add(Embedding(output_dim=32,input_dim=3800, input_length=380)) #數字list截長補短後都是380個數字
modelLSTM.add(Dropout(0.7)) 
modelLSTM .add(LSTM(32)) 
modelLSTM .add(Dense(units=1,activation='sigmoid')) 


modelLSTM .summary()

modelLSTM.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy']) 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 380, 32)           121600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 380, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 129,953
Trainable params: 129,953
Non-trainable params: 0
_________________________________________________________________


In [7]:
modelLSTM.fit(x_train, y, epochs=10, batch_size=100,verbose=2,validation_split=0.2)

C:\Users\HuanJian\Anaconda3\envs\tensorflow_gpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8000 samples, validate on 2000 samples
Epoch 1/10
 - 39s - loss: 0.6796 - accuracy: 0.5731 - val_loss: 0.6491 - val_accuracy: 0.6580
Epoch 2/10
 - 38s - loss: 0.5891 - accuracy: 0.6946 - val_loss: 0.5601 - val_accuracy: 0.7165
Epoch 3/10
 - 39s - loss: 0.4957 - accuracy: 0.7619 - val_loss: 0.5455 - val_accuracy: 0.7340
Epoch 4/10
 - 35s - loss: 0.4539 - accuracy: 0.7930 - val_loss: 0.5294 - val_accuracy: 0.7495
Epoch 5/10
 - 37s - loss: 0.4185 - accuracy: 0.8127 - val_loss: 0.5376 - val_accuracy: 0.7460
Epoch 6/10
 - 39s - loss: 0.3965 - accuracy: 0.8294 - val_loss: 0.5477 - val_accuracy: 0.7460
Epoch 7/10
 - 40s - loss: 0.3799 - accuracy: 0.8374 - val_loss: 0.5688 - val_accuracy: 0.7470
Epoch 8/10
 - 38s - loss: 0.3723 - accuracy: 0.8339 - val_loss: 0.5699 - val_accuracy: 0.7390
Epoch 9/10
 - 38s - loss: 0.3509 - accuracy: 0.8510 - val_loss: 0.5800 - val_accuracy: 0.7355
Epoch 10/10
 - 38s - loss: 0.3527 - accuracy: 0.8479 - val_loss: 0.5983 - val_accuracy: 0.7375


In [135]:
scores = modelLSTM.evaluate(x_test, Y_test,verbose=1)
scores[1]

90/90 [==============================] - 2s 23ms/step


0.8444444537162781